In [1]:
import xgboost as xgb
import numpy as np
from  sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.metrics import confusion_matrix,mean_squared_error
from sklearn.datasets import load_iris,load_digits,load_boston
from hyperopt import fmin, tpe, hp, partial

D:\software\anaconda\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
iris = load_iris()
x = iris['data']
y = iris['target']
x

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

## 数据分割

In [3]:
x_train_all,x_predcit,y_train_all,y_predict = train_test_split(x,y,test_size=0.1,random_state=100)
x_train,x_test,y_train,y_test = train_test_split(x_train_all,y_train_all,test_size=0.2,random_state=100)

## 数据格式转换

In [4]:
d_train = xgb.DMatrix(data=x_train,label=y_train)
d_test = xgb.DMatrix(data=x_test,label=y_test)
eval_list = [(d_train,'train'),(d_test,'test')]

In [15]:
# 自定义hyperopt的参数空间
space = {"max_depth": hp.randint("max_depth", 15),
         "n_estimators": hp.randint("n_estimators", 300),
         'learning_rate': hp.uniform('learning_rate', 1e-3, 5e-1),
         "subsample": hp.randint("subsample", 5),
         "min_child_weight": hp.randint("min_child_weight", 6),
         }

def argsDict_tranform(argsDict, isPrint=False):
    argsDict["max_depth"] = argsDict["max_depth"] + 5
    argsDict['n_estimators'] = argsDict['n_estimators'] + 150
    argsDict["learning_rate"] = argsDict["learning_rate"] * 0.02 + 0.05
    argsDict["subsample"] = argsDict["subsample"] * 0.1 + 0.5
    argsDict["min_child_weight"] = argsDict["min_child_weight"] + 1
    if isPrint:
        print(argsDict)
    else:
        pass

    return argsDict


In [16]:
def xgboost_factory(argsDict):
    argsDict = argsDict_tranform(argsDict)
    
    params = {'nthread': -1,  # 进程数
              'max_depth': argsDict['max_depth'],  # 最大深度
              'n_estimators': argsDict['n_estimators'],  # 树的数量
              'eta': argsDict['learning_rate'],  # 学习率
              'subsample': argsDict['subsample'],  # 采样数
              'min_child_weight': argsDict['min_child_weight'],  # 终点节点最小样本占比的和
              'objective': 'reg:linear',
              'gamma': 0,  # 是否后剪枝
              'colsample_bytree': 0.7,  # 样本列采样
              'alpha': 0,  # L1 正则化
              'lambda': 0,  # L2 正则化
              'scale_pos_weight': 0,  # 取值>0时,在数据不平衡时有助于收敛
              'seed': 100,  # 随机种子
              }
    params['eval_metric'] = ['rmse']

    xrf = xgb.train(params, d_train, params['n_estimators'], eval_list,early_stopping_rounds=100)

    return get_tranformer_score(xrf)

def get_tranformer_score(tranformer):
    
    xrf = tranformer
    dpredict = xgb.DMatrix(x_predcit)
    prediction = xrf.predict(dpredict, ntree_limit=xrf.best_ntree_limit)
  
    return mean_squared_error(y_predict, prediction)


In [17]:
# 开始使用hyperopt进行自动调参
algo = partial(tpe.suggest, n_startup_jobs=1)
best = fmin(xgboost_factory, space, algo=algo, max_evals=20, pass_expr_memo_ctrl=None)


[16:13:34] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:13:34] WARNING: ..\src\learner.cc:576:            
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:0.94139	test-rmse:0.65605              
[1]	train-rmse:0.89144	test-rmse:0.62456              
[2]	train-rmse:0.84545	test-rmse:0.59722              
[3]	train-rmse:0.80326	test-rmse:0.57384              
[4]	train-rmse:0.75224	test-rmse:0.53103              
[5]	train-rmse:0.71096	test-rmse:0.49750              
[6]	train-rmse:0.67953	test-rmse:0.48659              
[7]	train-rmse:0.65052	test-rmse:0.47658              
[8]	train-rmse:0.62195	test-rms

D:\software\anaconda\lib\site-packages\xgboost\core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(



[14]	train-rmse:0.48077	test-rmse:0.37938                                         
[15]	train-rmse:0.46697	test-rmse:0.37388                                         
[16]	train-rmse:0.44370	test-rmse:0.35306                                         
[17]	train-rmse:0.43286	test-rmse:0.35009                                         
[18]	train-rmse:0.42375	test-rmse:0.34831                                         
[19]	train-rmse:0.41655	test-rmse:0.34826                                         
[20]	train-rmse:0.41083	test-rmse:0.34643                                         
[21]	train-rmse:0.40519	test-rmse:0.34618                                         
[22]	train-rmse:0.39130	test-rmse:0.33592                                         
[23]	train-rmse:0.37913	test-rmse:0.32521                                         
[24]	train-rmse:0.37689	test-rmse:0.32784                                         
[25]	train-rmse:0.37569	test-rmse:0.32848                                         
[26]

D:\software\anaconda\lib\site-packages\xgboost\core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(



[20]	train-rmse:0.45123	test-rmse:0.39670                                         
[21]	train-rmse:0.44814	test-rmse:0.39806                                         
[22]	train-rmse:0.44186	test-rmse:0.39347                                         
[23]	train-rmse:0.43645	test-rmse:0.38845                                         
[24]	train-rmse:0.43596	test-rmse:0.39164                                         
[25]	train-rmse:0.43570	test-rmse:0.39382                                         
[26]	train-rmse:0.43341	test-rmse:0.39322                                         
[27]	train-rmse:0.43195	test-rmse:0.39332                                         
[28]	train-rmse:0.43271	test-rmse:0.39606                                         
[29]	train-rmse:0.43446	test-rmse:0.39610                                         
[30]	train-rmse:0.43623	test-rmse:0.39934                                         
[31]	train-rmse:0.43330	test-rmse:0.39622                                         
[32]

...

D:\software\anaconda\lib\site-packages\xgboost\core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(



[26]	train-rmse:0.41204	test-rmse:0.37503                                           
[27]	train-rmse:0.41051	test-rmse:0.37597                                           
[28]	train-rmse:0.41030	test-rmse:0.37727                                           
[29]	train-rmse:0.40993	test-rmse:0.37918                                           
[30]	train-rmse:0.41076	test-rmse:0.38089                                           
[31]	train-rmse:0.41052	test-rmse:0.38055                                           
[32]	train-rmse:0.41275	test-rmse:0.38267                                           
[33]	train-rmse:0.41369	test-rmse:0.38486                                           
[34]	train-rmse:0.41490	test-rmse:0.38543                                           
[35]	train-rmse:0.41665	test-rmse:0.38357                                           
[36]	train-rmse:0.41838	test-rmse:0.38577                                           
[37]	train-rmse:0.41968	test-rmse:0.38742                        

D:\software\anaconda\lib\site-packages\xgboost\core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(



[20]	train-rmse:0.43486	test-rmse:0.38234                                           
[21]	train-rmse:0.42995	test-rmse:0.38281                                           
[22]	train-rmse:0.42148	test-rmse:0.37546                                           
[23]	train-rmse:0.41791	test-rmse:0.37357                                           
[24]	train-rmse:0.41601	test-rmse:0.37392                                           
[25]	train-rmse:0.41459	test-rmse:0.37498                                           
[26]	train-rmse:0.41213	test-rmse:0.37502                                           
[27]	train-rmse:0.41057	test-rmse:0.37596                                           
[28]	train-rmse:0.41034	test-rmse:0.37724                                           
[29]	train-rmse:0.40995	test-rmse:0.37915                                           
[30]	train-rmse:0.41075	test-rmse:0.38085                                           
[31]	train-rmse:0.41050	test-rmse:0.38050                        

D:\software\anaconda\lib\site-packages\xgboost\core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(



In [18]:
RMSE = xgboost_factory(best)
print('best :', best)
print('best param after transform :')
argsDict_tranform(best,isPrint=True)
print('rmse of the best xgboost:', np.sqrt(RMSE))


[16:14:24] WARNING: c:\ci\xgboost-split_1638290375667\work\src\objective\regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:14:24] WARNING: ..\src\learner.cc:576: 
Parameters: { "n_estimators" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:0.93837	test-rmse:0.65436
[1]	train-rmse:0.88587	test-rmse:0.62132
[2]	train-rmse:0.83778	test-rmse:0.59290
[3]	train-rmse:0.79648	test-rmse:0.57469
[4]	train-rmse:0.75261	test-rmse:0.54244
[5]	train-rmse:0.70975	test-rmse:0.50741
[6]	train-rmse:0.67745	test-rmse:0.49627
[7]	train-rmse:0.64858	test-rmse:0.48788
[8]	train-rmse:0.61953	test-rmse:0.47364
[9]	train-rmse:0.59535	test-rmse:0.46579
[10]	train-rmse:0.56957	test-rmse:0.44763
[11]	train-rmse:0.54972	test-r

D:\software\anaconda\lib\site-packages\xgboost\core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(
